In [ ]:
from pathlib import Path
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from openai import OpenAI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Loading the documents

doc_path = ["/content/drive/MyDrive/01-mini hackathon/database.csv", "/content/drive/MyDrive/01-mini hackathon/Leetcode_Questions.csv", "/content/drive/MyDrive/01-mini hackathon/leetcode.csv"]
docs = []
for doc_file in doc_path:
  file_path = Path(doc_file)
  print(doc_file)
  try:
    if doc_file.endswith (".csv"):
      loader = CSVLoader(file_path)
    elif doc_file.endswith(".pdf"):
      loader = PyPDFLoader(file_path)
    elif doc_file.endswith(".docx"):
      loader = Docx2txtLoader(file_path)
    elif doc_file.endswith(".txt") or doc_file.endswith(".md"):
      loader = TextLoader(file_path)
    else:
      print(f"DocumentType {doc_file.type} not supported.")
      continue
    docs.extend(loader.load())
  except Exception as e:
    print(f"Error loading document {doc_file}: {e}")


/content/drive/MyDrive/01-mini hackathon/database.csv
/content/drive/MyDrive/01-mini hackathon/Leetcode_Questions.csv
/content/drive/MyDrive/01-mini hackathon/leetcode.csv


In [ ]:
print(docs[:50])

[Document(metadata={'source': '/content/drive/MyDrive/01-mini hackathon/database.csv', 'row': 0}, page_content=': 0\nQuestion: 1\nAnswer: class Solution {\n public:\n  vector<int> twoSum(vector<int>& nums, int target) {\n    unordered_map<int, int> numToIndex;\n\n    for (int i = 0; i < nums.size(); ++i) {\n      if (const auto it = numToIndex.find(target - nums[i]);\n          it != cend(numToIndex))\n        return {it->second, i};\n      numToIndex[nums[i]] = i;\n    }\n\n    throw;\n  }\n};'), Document(metadata={'source': '/content/drive/MyDrive/01-mini hackathon/database.csv', 'row': 1}, page_content=': 1\nQuestion: 2\nAnswer: class Solution {\n public:\n  ListNode* addTwoNumbers(ListNode* l1, ListNode* l2) {\n    ListNode dummy(0);\n    ListNode* curr = &dummy;\n    int carry = 0;\n\n    while (l1 || l2 || carry) {\n      if (l1 != nullptr) {\n        carry += l1->val;\n        l1 = l1->next;\n      }\n      if (l2 != nullptr) {\n        carry += l2->val;\n        l2 = l2->next;\

In [ ]:
#Splitting the documents

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 5000,
                                               chunk_overlap = 1000)

documents_chunks = text_splitter.split_documents(docs)


In [ ]:
vector_db = Chroma.from_documents(documents = documents_chunks,
                                  embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2"))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
client = OpenAI(
    api_key="Your_API_KEY",
    base_url="https://api.aimlapi.com/v1",
)

chat_history = []

while True:
    user_input = input("You: ")

    # Retrieve the most relevant document from the vector database
    result = vector_db.similarity_search(user_input, k=1)
    retrieved_content = result[0].page_content if result else "No relevant document found."

    # Append the user input and retrieved document to the chat history
    chat_history.append({"role": "user", "content": user_input})
    # chat_history.append({"role": "assistant", "content": retrieved_content})
    messages = chat_history.copy()
    # Sending the conversation history to OpenAI API
    response = client.chat.completions.create(
        model="o1-preview",
        messages=messages
        ,max_tokens = 50000
    )

    # Get and print the assistant's reply
    assistant_reply = response.choices[0].message.content
    print(f"Assistant: {assistant_reply}")

    # Append assistant's reply to the chat history for context in the next loop
    chat_history.append({"role": "assistant", "content": assistant_reply})



# Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

# You may assume that each input would have exactly one solution, and you may not use the same element twice.

# You can return the answer in any order.

You: A positive integer is an  N N-subtractorization if it is a prime number that can be expressed as the difference of two primes that are both at most  N N. For example, the complete set of  5 5-subtractorizations is  { 2 , 3 } {2,3} because  5 − 3 = 2 5−3=2 and  5 − 2 = 3 5−2=3. For a given  N N, how many  N N-subtractorizations are there?
Assistant: To solve this problem, we need to understand the definition of an "**NN-subtractorization**." Given **N**, the **N-subtractorizations** are the set of primes **p** such that both **p** and **N - p** are primes less than or equal to **N**.

**Understanding the Definition:**

Given the example in the question for **N = 5**:
- The primes less than or equal to 5 are **2, 3, 5**.
- Computing **N - p** for each prime:
  - **5 - 2 = 3**, which is prime and ≤ 5.
  - **5 - 3 = 2**, which is prime and ≤ 5.
  - **5 - 5 = 0**, which is not prime.
- The primes **2** and **3** satisfy the condition, so the **5-subtractorizations** are **{2, 3}**.

**

KeyboardInterrupt: Interrupted by user